In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/ComputerVision/Covid19-Detector/

Mounted at /content/gdrive
/content/gdrive/MyDrive/ComputerVision/Covid19-Detector


In [2]:
!pip install import-ipynb
import import_ipynb

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-py3-none-any.whl size=2975 sha256=0d1e60dd30aa431e0da0b47e9c150d4331dfa355250324d0024c47551a58b798
  Stored in directory: /root/.cache/pip/wheels/b1/5e/dc/79780689896a056199b0b9f24471e3ee184fbd816df355d5f0
Successfully built import-ipynb


In [3]:
from data_loader import DatasetLoader
from simplepreprocessor import SimplePreprocessor
from imagetoarraypreprocessor import ImageToArrayPreprocessor
from neural_network_architecture import  LeNet, MiniVGGNet, ShallowNet

importing Jupyter notebook from data_loader.ipynb
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
importing Jupyter notebook from simplepreprocessor.ipynb
importing Jupyter notebook from imagetoarraypreprocessor.ipynb
importing Jupyter notebook from neural_network_architecture.ipynb


In [4]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import SGD
import matplotlib.pyplot as plt
import numpy as np

In [5]:

WIDTH = 224
HEIGHT = 224

sp = SimplePreprocessor(WIDTH, HEIGHT)
iap = ImageToArrayPreprocessor()

In [6]:
dl = DatasetLoader(preprocessors=[sp, iap])

In [7]:
(data, labels) = dl.load_data()

In [8]:
(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size=0.25, random_state=412)

In [9]:
trainY

array(['Negative', 'Negative', 'Negative', ..., 'Negative', 'Negative',
       'Negative'], dtype='<U8')

In [10]:
# convert the labels from integers to vectors
#trainY = LabelBinarizer().fit_transform(trainY)
#testY = LabelBinarizer().fit_transform(testY)

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
le = LabelEncoder().fit(trainY)
trainY = to_categorical(le.transform(trainY), 2)
testY = to_categorical(le.transform(testY), 2)

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# initialize an our data augmenter 
aug = ImageDataGenerator(
		rotation_range=20,
		zoom_range=0.15,
		width_shift_range=0.2,
		height_shift_range=0.2,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

In [12]:
print("Compiling MiniVGGNet  Model")


INIT_LR = 1e-3
BS = 32
EPOCHS = 100
opt = SGD(lr=INIT_LR, momentum=0.9, decay=INIT_LR / EPOCHS)
model = MiniVGGNet.build(width=WIDTH, height=HEIGHT, depth=3, classes=2)
model.compile(loss= "binary_crossentropy", optimizer=opt,
metrics=["accuracy"])

# train the network
print("[INFO] training network...")
H = model.fit(x=aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY),
	steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS)


Compiling MiniVGGNet  Model


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


[INFO] training network...
Epoch 1/100
124/124 [==============================] - 48s 317ms/step - loss: 0.7323 - accuracy: 0.6612 - val_loss: 0.3905 - val_accuracy: 0.9842
Epoch 2/100
124/124 [==============================] - 38s 305ms/step - loss: 0.4459 - accuracy: 0.8708 - val_loss: 1.8374 - val_accuracy: 0.6594
Epoch 3/100
124/124 [==============================] - 38s 308ms/step - loss: 0.2975 - accuracy: 0.9458 - val_loss: 6.3139 - val_accuracy: 0.0810
Epoch 4/100
124/124 [==============================] - 38s 310ms/step - loss: 0.2160 - accuracy: 0.9695 - val_loss: 4.7662 - val_accuracy: 0.5071
Epoch 5/100
124/124 [==============================] - 39s 313ms/step - loss: 0.1629 - accuracy: 0.9798 - val_loss: 2.0318 - val_accuracy: 0.8095
Epoch 6/100
124/124 [==============================] - 42s 338ms/step - loss: 0.1381 - accuracy: 0.9770 - val_loss: 1.9812 - val_accuracy: 0.8245
Epoch 7/100
124/124 [==============================] - 38s 302ms/step - loss: 0.1122 - accuracy: 

In [13]:
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=BS)
print(classification_report(testY.argmax(axis=1),
predictions.argmax(axis=1), target_names=le.classes_))

[INFO] evaluating network...
              precision    recall  f1-score   support

    Negative       0.99      1.00      0.99      1312
    positive       0.89      0.38      0.53        21

    accuracy                           0.99      1333
   macro avg       0.94      0.69      0.76      1333
weighted avg       0.99      0.99      0.99      1333

